In [1]:
import os
import features
import numpy as np
import skimage.io
import csv
import re
from collections import namedtuple
from sklearn.externals import joblib
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
Sample = namedtuple('Sample', ['path','label','vector'])

def map_file_path_to_number(file_path):
    for label in LABELS:
        if label in file_path:
            return labels_to_number[label]
    return -1

def map_row_to_sample(csv_row):
    return Sample(
        path=row[0],
        label=map_file_path_to_number(row[0]),
        vector=[float(x) for x in row[2:]]
    )

vectors_dump_file = open('gaze_vectors_with_failures.csv', 'rb')
vectors_and_failures = list(csv.reader(vectors_dump_file))

LABELS = ['top_left', 'top_right', 'bot_left', 'bot_right']
labels_to_number = dict(zip(LABELS, range(len(LABELS))))
samples = [map_row_to_sample(row) for row in vectors_and_failures if not row[1]]

# Samples with multiple Faces

In [3]:
paths = [s.path for s in samples]
detections_per_file_counts = [[x,paths.count(x)] for x in set(paths)]
multiple_detections = [x for x in detections_per_file_counts if x[1] > 1]

In [4]:
def path_to_participant_label(file_path):
    p = re.compile('p\d+')
    str_label = p.search(file_path).group() # "p<x>"
    return int(str_label[1:]) # "p<x>" -> <x>

contaminated_p_folders = set([path_to_participant_label(x[0]) for x in multiple_detections])

In [5]:
print "Numer of multiple detections: ", len(multiple_detections)
print "Particpant folders with multiple detections: ", contaminated_p_folders

Numer of multiple detections:  184
Particpant folders with multiple detections:  set([1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 15, 17, 19, 21])


In [6]:
multiple_detections

[['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p15/middle_left/frame_320_top_right_straight.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p15/front_left/frame_179_bot_left_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p7/front_center/frame_316_bot_left_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p7/front_center/frame_304_bot_right_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p7/front_center/frame_305_bot_right_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p19/front_right/frame_167_top_right_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oct-23/p5/middle_left/frame_172_top_right_natural.jpg',
  2],
 ['/Users/vt/Code/Internal/lab-gaze-tracker/training_data/company-day-oc

In [10]:
joblib.dump(set(x[0] for x in multiple_detections), 'samples-with-multi-detections.pkl');